In [ ]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 855.1 kB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=39796dbb8f85b02e21c9605aeeabbf81b96434fc54b91755b1b621d15d675e51
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
import requests
# import cloudscraper
import time
import random
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import json
import warnings
import ast
from datetime import date
from datetime import datetime, timedelta
import pytz
import re
timezone = pytz.timezone('Asia/Jakarta')
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

In [ ]:
with open('drive/MyDrive/Dataset/Berita RUU/20240918_berita ruu semester 1 2024.pkl', 'rb') as handle:
    all_news = pickle.load(handle)

In [ ]:
df_news = pd.DataFrame(all_news)

In [ ]:
df_news.sample(10)

,title,description,published date,url,publisher
173,"Banyak Masyarakat Adat Tak Punya KTP, Pemerint...","Banyak Masyarakat Adat Tak Punya KTP, Pemerint...","Tue, 23 Jan 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiwwFBV...,"{'href': 'https://nasional.kompas.com', 'title..."
2610,"Polemik RUU Penyiaran, Komisi I DPR Minta Peme...","Polemik RUU Penyiaran, Komisi I DPR Minta Peme...","Mon, 27 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMixAFBV...,"{'href': 'https://nasional.kompas.com', 'title..."
2858,"Polisi Ikut Awasi Ruang Siber, Penyadapan Jadi...","Polisi Ikut Awasi Ruang Siber, Penyadapan Jadi...","Wed, 29 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMipAFBV...,"{'href': 'https://www.kompas.id', 'title': 'ko..."
2660,"RUU Penyiaran Membungkam Kebebasan Pers, Berik...","RUU Penyiaran Membungkam Kebebasan Pers, Berik...","Mon, 27 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMitwFBV...,"{'href': 'https://kepritoday.com', 'title': 'K..."
2634,Jurnalis Se-Kota Bontang Tolak Pembahasan RUU ...,Jurnalis Se-Kota Bontang Tolak Pembahasan RUU ...,"Sun, 26 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMimwFBV...,"{'href': 'https://klikkaltim.com', 'title': 'K..."
31,"KJP Plus 492 Siswa Dicabut pada 2023, Penyebab...","KJP Plus 492 Siswa Dicabut pada 2023, Penyebab...","Fri, 05 Jan 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMisAFBV...,"{'href': 'https://tekno.tempo.co', 'title': 'T..."
446,"Angkat Kartu Merah di Aksi Kamisan, Ini Profil...","Angkat Kartu Merah di Aksi Kamisan, Ini Profil...","Sat, 17 Feb 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiwgFBV...,"{'href': 'https://seleb.tempo.co', 'title': 'S..."
2760,"Tolak RUU MK Dan RUU Penyiaran, Banteng Mulai ...","Tolak RUU MK Dan RUU Penyiaran, Banteng Mulai ...","Tue, 28 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMilAFBV...,"{'href': 'https://rm.id', 'title': 'RM.ID'}"
1374,"Khofifah Akui Bertemu Elite PDIP, Bahas Ekonom...","Khofifah Akui Bertemu Elite PDIP, Bahas Ekonom...","Tue, 09 Apr 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiwwFBV...,"{'href': 'https://www.cnnindonesia.com', 'titl..."
2853,Rapat Paripurna Pengambilan Keputusan Empat RU...,Rapat Paripurna Pengambilan Keputusan Empat RU...,"Tue, 28 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMi6gFBV...,"{'href': 'https://nasional.sindonews.com', 'ti..."


In [ ]:
df_news.iloc[2406]['publisher']

{'href': 'https://news.detik.com', 'title': 'detikNews'}

In [ ]:
df_news['publisher_link'] = df_news['publisher'].apply(lambda x:x['href'])
df_news['publisher_title'] = df_news['publisher'].apply(lambda x:x['title'])

In [ ]:
df_news['published date'] = pd.to_datetime(df_news['published date'], format='%a, %d %b %Y %H:%M:%S %Z')

In [ ]:
df_news.shape

(3999, 7)

In [ ]:
df_news.drop_duplicates(['title','publisher_title']).shape

(2159, 7)

In [ ]:
df_news = df_news.drop_duplicates(['title','publisher_title']).reset_index(drop=True)

In [ ]:
df_news.loc[df_news['title'].str.contains('RUU tentang')]

,title,description,published date,url,publisher,publisher_link,publisher_title
305,Konsultasi Publik RUU tentang Pengelolaan Keka...,Konsultasi Publik RUU tentang Pengelolaan Keka...,2024-02-29 08:00:00+00:00,https://news.google.com/rss/articles/CBMi1wFBV...,"{'href': 'https://www.djkn.kemenkeu.go.id', 't...",https://www.djkn.kemenkeu.go.id,Direktorat Jenderal Kekayaan Negara
678,Presiden Polandia veto amandemen RUU tentang p...,Presiden Polandia veto amandemen RUU tentang p...,2024-03-30 07:00:00+00:00,https://news.google.com/rss/articles/CBMirgFBV...,"{'href': 'https://www.antaranews.com', 'title'...",https://www.antaranews.com,ANTARA
1665,Pemerintah Taiwan akan Kembalikan RUU tentang ...,Pemerintah Taiwan akan Kembalikan RUU tentang ...,2024-06-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMiVkFVX...,"{'href': 'https://www.voaindonesia.com', 'titl...",https://www.voaindonesia.com,VOA Indonesia
2001,Pandangan Fraksi PKS DPR RI terhadap RUU tenta...,Pandangan Fraksi PKS DPR RI terhadap RUU tenta...,2024-06-19 07:00:00+00:00,https://news.google.com/rss/articles/CBMi1AJBV...,"{'href': 'https://fraksi.pks.id', 'title': 'Fr...",https://fraksi.pks.id,Fraksi PKS


In [ ]:
df_news.iloc[2001]['title']

'Pandangan Fraksi PKS DPR RI terhadap RUU tentang Pengesahan Memorandum Saling Pengertian antara Kementerian Pertahanan Republik Indonesia dan Kementerian Pertahanan Persatuan Emirat Arab mengenai Kerja Sama di Bidang Pertahanan - Fraksi PKS'

In [ ]:
df_news.loc[df_news['title'].apply(lambda x:'tapera' in x.lower())]

,title,description,published date,url,publisher,publisher_link,publisher_title
1562,"Jejak Aturan Tapera di DPR, Disetujui Semua Fr...","Jejak Aturan Tapera di DPR, Disetujui Semua Fr...",2024-05-29 07:00:00+00:00,https://news.google.com/rss/articles/CBMiywFBV...,"{'href': 'https://www.cnnindonesia.com', 'titl...",https://www.cnnindonesia.com,CNN Indonesia
1636,"Perjalanan Tapera, Digulirkan Saat Era SBY dan...","Perjalanan Tapera, Digulirkan Saat Era SBY dan...",2024-05-31 07:00:00+00:00,https://news.google.com/rss/articles/CBMiywFBV...,"{'href': 'https://www.kompas.com', 'title': 'K...",https://www.kompas.com,KOMPAS.com
1663,"Dulu Dukung saat Pembahasan RUU, PKS Kini Tola...","Dulu Dukung saat Pembahasan RUU, PKS Kini Tola...",2024-05-30 07:00:00+00:00,https://news.google.com/rss/articles/CBMi0wFBV...,"{'href': 'https://solo.tribunnews.com', 'title...",https://solo.tribunnews.com,Tribun Solo
1690,Inikah Pengalihan Isu? Kasus Vina Cirebon di A...,Inikah Pengalihan Isu? Kasus Vina Cirebon di A...,2024-06-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMimAJBV...,"{'href': 'https://jogjakartanews.com', 'title'...",https://jogjakartanews.com,Jogjakartanews
1884,"Basuki Mengaku Sesali Tapera, Akan Ikuti DPR d...","Basuki Mengaku Sesali Tapera, Akan Ikuti DPR d...",2024-06-08 07:00:00+00:00,https://news.google.com/rss/articles/CBMiswFBV...,"{'href': 'https://kumparan.com', 'title': 'kum...",https://kumparan.com,kumparan.com
1896,Aksi HMI Tolak Tapera hingga RUU Penyiaran - d...,Aksi HMI Tolak Tapera hingga RUU Penyiaran de...,2024-06-10 07:00:00+00:00,https://news.google.com/rss/articles/CBMijgFBV...,"{'href': 'https://news.detik.com', 'title': 'd...",https://news.detik.com,detikNews
1920,Mahasiswa di Tegal Gelar Aksi Demo Tuntut Tape...,Mahasiswa di Tegal Gelar Aksi Demo Tuntut Tape...,2024-06-11 07:00:00+00:00,https://news.google.com/rss/articles/CBMiugFBV...,"{'href': 'https://radartegal.disway.id', 'titl...",https://radartegal.disway.id,Radar Tegal
1963,"Mahasiswa Unjuk Rasa Tolak Tapera, RUU Penyiar...","Mahasiswa Unjuk Rasa Tolak Tapera, RUU Penyiar...",2024-06-15 07:00:00+00:00,https://news.google.com/rss/articles/CBMingFBV...,"{'href': 'https://kepri.batampos.co.id', 'titl...",https://kepri.batampos.co.id,Kepri
2094,Buruh Bakal Gelar Demo Tolak RUU Tapera Kamis ...,Buruh Bakal Gelar Demo Tolak RUU Tapera Kamis ...,2024-06-26 07:00:00+00:00,https://news.google.com/rss/articles/CBMiekFVX...,"{'href': 'https://infoka.id', 'title': 'infoka'}",https://infoka.id,infoka
2108,Mahasiswa HMI Cabang Jember Lakukan Unjuk Rasa...,Mahasiswa HMI Cabang Jember Lakukan Unjuk Rasa...,2024-06-27 07:00:00+00:00,https://news.google.com/rss/articles/CBMiywFBV...,"{'href': 'https://radarjember.jawapos.com', 't...",https://radarjember.jawapos.com,Radar Jember


In [ ]:
df_news.loc[df_news['title'].apply(lambda x:'ruu koperasi' in x.lower())]

,title,description,published date,url,publisher,publisher_link,publisher_title
1141,"Jelang Jokowi Lengser, Apa Kabar RUU Koperasi?...","Jelang Jokowi Lengser, Apa Kabar RUU Koperasi?...",2024-05-17 07:00:00,https://news.google.com/rss/articles/CBMiugFBV...,"{'href': 'https://ekonomi.bisnis.com', 'title'...",https://ekonomi.bisnis.com,Bisnis.com


In [ ]:
# Function to extract RUU topic
def extract_ruu_topic(title):
    match = re.search(r'\bRUU ([A-Z][a-zA-Z]*)\b', title)
    if match:
        return f"RUU {match.group(1)}"
    return np.nan

# Apply the function to the 'title' column to create 'ruu_topic' column
df_news['ruu_topic'] = df_news['title'].apply(extract_ruu_topic)

In [ ]:
df_news.sample(10)

,title,description,published date,url,publisher,publisher_link,publisher_title,ruu_topic
1902,"Respons soal RUU Penyiaran, KPID Jakarta: Pro ...","Respons soal RUU Penyiaran, KPID Jakarta: Pro ...",2024-06-11 07:00:00+00:00,https://news.google.com/rss/articles/CBMirAFBV...,"{'href': 'https://nasional.tempo.co', 'title':...",https://nasional.tempo.co,Nasional Tempo,RUU Penyiaran
255,"Perbedaan DPR, DPD & DPRD Beserta Cara Cek Rea...","Perbedaan DPR, DPD & DPRD Beserta Cara Cek Rea...",2024-02-16 08:00:00+00:00,https://news.google.com/rss/articles/CBMirgFBV...,"{'href': 'https://www.detik.com', 'title': 'de...",https://www.detik.com,detikcom,NaN
428,Menpan RB Sebut Wacana Cuti Ayah Masih Tahap P...,Menpan RB Sebut Wacana Cuti Ayah Masih Tahap P...,2024-03-14 07:00:00+00:00,https://news.google.com/rss/articles/CBMiswFBV...,"{'href': 'https://www.tribunnews.com', 'title'...",https://www.tribunnews.com,Tribunnews.com,RUU ASN
618,DPR Sahkan RUU Daerah Khusus Jakarta jadi Unda...,DPR Sahkan RUU Daerah Khusus Jakarta jadi Unda...,2024-03-28 07:00:00+00:00,https://news.google.com/rss/articles/CBMimAFBV...,"{'href': 'https://www.rri.co.id', 'title': 'rr...",https://www.rri.co.id,rri.co.id,RUU Daerah
1184,Joe Biden: RUU Penerbangan hingga Tarif Impor ...,Joe Biden: RUU Penerbangan hingga Tarif Impor ...,2024-05-18 07:00:00+00:00,https://news.google.com/rss/articles/CBMilAFBV...,"{'href': 'https://dunia.tempo.co', 'title': 'D...",https://dunia.tempo.co,Dunia Tempo,RUU Penerbangan
1600,Melihat Lagi RUU Polri soal Mekanisme Putus Ak...,Melihat Lagi RUU Polri soal Mekanisme Putus Ak...,2024-05-30 07:00:00+00:00,https://news.google.com/rss/articles/CBMipwFBV...,"{'href': 'https://kumparan.com', 'title': 'kum...",https://kumparan.com,kumparan.com,RUU Polri
1915,Membentuk Presidential Club atau RUU Lembaga K...,Membentuk Presidential Club atau RUU Lembaga K...,2024-06-11 07:00:00+00:00,https://news.google.com/rss/articles/CBMi1wFBV...,"{'href': 'https://www.kompasiana.com', 'title'...",https://www.kompasiana.com,Kompasiana.com,RUU Lembaga
11,Menkeu : Belanja Negara Terakselerasi Naik - K...,Menkeu : Belanja Negara Terakselerasi Naik Ke...,2024-01-03 08:00:00+00:00,https://news.google.com/rss/articles/CBMiqwFBV...,"{'href': 'https://www.kemenkeu.go.id', 'title'...",https://www.kemenkeu.go.id,Kementerian Keuangan,NaN
2081,BSSN tegaskan RUU keamanan siber mendesak demi...,BSSN tegaskan RUU keamanan siber mendesak demi...,2024-06-26 07:00:00+00:00,https://news.google.com/rss/articles/CBMirAFBV...,"{'href': 'https://www.antaranews.com', 'title'...",https://www.antaranews.com,ANTARA,NaN
1770,Dasco: RUU TNI dan Polri Tetap Terbatas - rri....,Dasco: RUU TNI dan Polri Tetap Terbatas rri.c...,2024-06-04 07:00:00+00:00,https://news.google.com/rss/articles/CBMiggFBV...,"{'href': 'https://www.rri.co.id', 'title': 'rr...",https://www.rri.co.id,rri.co.id,RUU TNI


In [ ]:
df_news.groupby(['ruu_topic'],as_index=False).agg(count=('title','count')).sort_values('count',ascending=False).head(10)

,ruu_topic,count
101,RUU Penyiaran,427
21,RUU DKJ,124
115,RUU Polri,76
102,RUU Perampasan,60
126,RUU TNI,53
35,RUU KIA,44
61,RUU MK,38
44,RUU Kementerian,37
23,RUU Desa,33
39,RUU Kabupaten,22


In [ ]:
df_news.loc[df_news['title'].str.contains('RUU Perampasan')].iloc[0]['title']

'Mahfud Sindir RUU Perampasan Aset Mandek di DPR: 6 Bulan Tak Ada Respons - detikNews'

# gemini monthly

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata
from IPython.display import Markdown

In [ ]:
#give gemini api to google generative AI
geminiKey = userdata.get('geminiKey')
genai.configure(api_key = geminiKey)

In [ ]:
generation_config = {
    "temperature":0,
    "top_p":1,
    "top_k":1,
    "max_output_tokens":400
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [ ]:
# get the model
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [ ]:
df_news

,title,description,published date,url,publisher,publisher_link,publisher_title
0,RUU DKJ Tak Sesuai Harapan Masyarakat Betawi -...,RUU DKJ Tak Sesuai Harapan Masyarakat Betawi ...,2024-01-01 08:00:00+00:00,https://news.google.com/rss/articles/CBMilgFBV...,"{'href': 'https://rmol.id', 'title': 'RMOL ID'}",https://rmol.id,RMOL ID
1,Menkeu : Pembiayaan Anggaran 2023 Turun Drop -...,Menkeu : Pembiayaan Anggaran 2023 Turun Drop ...,2024-01-02 08:00:00+00:00,https://news.google.com/rss/articles/CBMipwFBV...,"{'href': 'https://www.kemenkeu.go.id', 'title'...",https://www.kemenkeu.go.id,Kementerian Keuangan
2,"Dialog lintas agama, PSI perjuangkan kemudahan...","Dialog lintas agama, PSI perjuangkan kemudahan...",2024-01-02 08:00:00+00:00,https://news.google.com/rss/articles/CBMiqAFBV...,"{'href': 'https://www.antaranews.com', 'title'...",https://www.antaranews.com,ANTARA
3,Eks Hakim MK Arsyad Sanusi Meninggal Dunia - C...,Eks Hakim MK Arsyad Sanusi Meninggal Dunia CN...,2024-01-01 08:00:00+00:00,https://news.google.com/rss/articles/CBMiqgFBV...,"{'href': 'https://www.cnnindonesia.com', 'titl...",https://www.cnnindonesia.com,CNN Indonesia
4,MAKI Heran Harun Masiku Sembunyi Lama Padahal ...,MAKI Heran Harun Masiku Sembunyi Lama Padahal ...,2024-01-02 08:00:00+00:00,https://news.google.com/rss/articles/CBMitgFBV...,"{'href': 'https://news.detik.com', 'title': 'd...",https://news.detik.com,detikNews
...,...,...,...,...,...,...,...
2154,"HUT Bhayangkara ke-78, Amnesty International: ...","HUT Bhayangkara ke-78, Amnesty International: ...",2024-07-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMitwFBV...,"{'href': 'https://metro.tempo.co', 'title': 'M...",https://metro.tempo.co,Metro Tempo
2155,Kepala Bappenas paparkan urgensi penyusunan RU...,Kepala Bappenas paparkan urgensi penyusunan RU...,2024-07-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMiqAFBV...,"{'href': 'https://www.antaranews.com', 'title'...",https://www.antaranews.com,ANTARA
2156,"HUT Bhayangkara, Koalisi Sipil Tolak RUU Polri...","HUT Bhayangkara, Koalisi Sipil Tolak RUU Polri...",2024-07-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMijgFBV...,"{'href': 'https://kbr.id', 'title': 'KBR'}",https://kbr.id,KBR
2157,Aksi Tolak Tapera di Gorontalo Diwarnai Baku H...,Aksi Tolak Tapera di Gorontalo Diwarnai Baku H...,2024-07-01 07:00:00+00:00,https://news.google.com/rss/articles/CBMihgFBV...,"{'href': 'https://go-pena.id', 'title': 'Go-pe...",https://go-pena.id,Go-pena


In [ ]:

# Convert 'published date' to timezone-naive
df_news['published date'] = df_news['published date'].dt.tz_localize(None)

# Initialize an empty dictionary to store the dataframes
d_dfs = {}

# Loop through the months from January to June in 2024
for month in range(1, 7):
    # Create a datetime object for the start and end of the month
    start_date = pd.Timestamp(year=2024, month=month, day=1)
    end_date = start_date + pd.DateOffset(months=1)

    # Filter the dataframe for the current month
    monthly_df = df_news[(df_news['published date'] >= start_date) & (df_news['published date'] < end_date)]

    # Create a key for the dictionary in the format 'Month Year'
    key = start_date.strftime('%B %Y')

    # Store the filtered dataframe in the dictionary
    d_dfs[key] = monthly_df

# Now d_dfs will contain dataframes for each month from January to June 2024

In [ ]:
d_dfs.keys()

dict_keys(['January 2024', 'February 2024', 'March 2024', 'April 2024', 'May 2024', 'June 2024'])

In [ ]:
d_dfs['January 2024'].loc[d_dfs['January 2024']['title'].apply(lambda x:'polri' in x.lower())]

,title,description,published date,url,publisher,publisher_link,publisher_title
28,Jokowi Sudah Teken Kenaikan Gaji ASN hingga TN...,Jokowi Sudah Teken Kenaikan Gaji ASN hingga TN...,2024-01-08 08:00:00,https://news.google.com/rss/articles/CBMimAFBV...,"{'href': 'https://news.detik.com', 'title': 'd...",https://news.detik.com,detikNews


In [ ]:
for month, df in d_dfs.items():
    print(len(d_dfs[month]))

140
175
379
182
788
480


In [ ]:
df_news.loc[df_news['title']=='Aturan Intelkam di Draf RUU Polri Dinilai Tumpang Tindih dengan Tugas BIN dan BAIS TNI - Nasional Kompas.com']

,title,description,published date,url,publisher,publisher_link,publisher_title
1664,Aturan Intelkam di Draf RUU Polri Dinilai Tump...,Aturan Intelkam di Draf RUU Polri Dinilai Tump...,2024-06-01 07:00:00,https://news.google.com/rss/articles/CBMixAFBV...,"{'href': 'https://nasional.kompas.com', 'title...",https://nasional.kompas.com,Nasional Kompas.com


In [ ]:
d_files = {}

for month, df in d_dfs.items():
    # Path to the PDF file in the 'files' folder
    file_path = f"output_{month}.csv"

    d_dfs[month][['title']].to_csv(file_path, index=False)

    d_files[month] = genai.upload_file(path=file_path, display_name=month)

    # Confirm upload
    print(f"Uploaded file '{d_files[month].display_name}' as: {d_files[month].uri}")

Uploaded file 'January 2024' as: https://generativelanguage.googleapis.com/v1beta/files/zn7s12p8ed5v
Uploaded file 'February 2024' as: https://generativelanguage.googleapis.com/v1beta/files/d2isjwh1dzn3
Uploaded file 'March 2024' as: https://generativelanguage.googleapis.com/v1beta/files/kqtlihkkgc8u
Uploaded file 'April 2024' as: https://generativelanguage.googleapis.com/v1beta/files/ozsuzhdla5y7
Uploaded file 'May 2024' as: https://generativelanguage.googleapis.com/v1beta/files/hsdmz1qz3218
Uploaded file 'June 2024' as: https://generativelanguage.googleapis.com/v1beta/files/v79d1mhordjt


In [ ]:
d_responses = {}
for month, df in d_dfs.items():
    print(month)
    response = model.generate_content([d_files[month], f"""This is a csv of Indonesian news about RUU (rancangan undang-undang) in {month}, I want you to extract the RUU mentioned during this specific month.
    Please return it in a list text separated with comma and no any other texts. For example: RUU Perampasan Aset, RUU Penyiaran, ...
    """])

    d_responses[month] = response.text
    time.sleep(random.randint(5,20))


January 2024
February 2024
March 2024
April 2024
May 2024
June 2024


In [ ]:
d_responses

{'January 2024': 'RUU DKJ, RUU Perampasan Aset, RUU Penyiaran, RUU Masyarakat Adat, RUU PPRT, RUU Desa, RUU PHN, RUU Kesehatan, RUU BUMN, RUU Pendanaan Politik, RUU Lembaga Kepresidenan, RUU KSDAHE, RUU Administrasi Pemerintahan, RUU Aset Daerah, RUU Udara Bersih, RUU Sisdiknas, RUU Konservasi, RUU tentang Pendanaan Partai Politik, RUU BHP, RUU Perlayaran, RUU ITE, RUU Daerah Khusus Jakarta \n',
 'February 2024': 'RUU Masyarakat Adat, RUU Desa, RUU DKJ, RUU Perampasan Aset, RUU Penyiaran, RUU PHN, RUU EBET, RUU Pilkada, RUU PPRT, RUU Pelestarian dan perlindungan Adat Budaya Kerajaan Nusantara, RUU tentang Pengelolaan Kekayaan Negara, RUU ASN, RUU Kesehatan \n',
 'March 2024': 'RUU DKJ, RUU Desa, RUU Masyarakat Adat, RUU PPRT, RUU Penyiaran, RUU EBET, RUU KIA, RUU Ketenagakerjaan, RUU Perkoperasian, RUU Paten, RUU KUHP, RUU Ombudsman, RUU Kepariwisataan, RUU Pertembakauan, RUU Kesehatan, RUU Pilkada, RUU Sisdiknas, RUU Masyarakat Adat, RUU Masyarakat Adat, RUU Masyarakat Adat, RUU Masya

In [ ]:
for month, df in d_dfs.items():
    # if month == 'April 2024':
        print(month)
        response = model.generate_content([d_files[month], f"""This is a csv of Indonesian news about RUU (rancangan undang-undang) in {month}, I want you to extract the RUU mentioned during this specific month.
        Please only return RUU in Indonesia only, not other country. Please return it in a list text separated with semicolon and no any other texts and NO DUPLICATES. For example: RUU Perampasan Aset; RUU Penyiaran; ...
        """])

        d_responses[month] = response.text
        time.sleep(random.randint(5,20))

April 2024


In [ ]:
d_responses

{'January 2024': 'RUU DKJ; RUU Perampasan Aset; RUU Penyiaran; RUU Masyarakat Adat; RUU PPRT; RUU Desa; RUU Aset Daerah; RUU Lembaga Kepresidenan; RUU KSDAHE; RUU Pendanaan Politik; RUU Perlayaran; RUU BHP; RUU Kesehatan; RUU BUMN; RUU PHN; RUU Administrasi Pemerintahan; RUU Udara Bersih; RUU Sisdiknas; RUU Daerah Khusus Jakarta; RUU Konservasi; RUU tentang Pendanaan Partai Politik \n',
 'February 2024': 'RUU Masyarakat Adat; RUU Desa; RUU DKJ; RUU Perampasan Aset; RUU Penyiaran; RUU PHN; RUU EBET; RUU Pilkada; RUU PPRT; RUU Pelestarian dan perlindungan Adat Budaya Kerajaan Nusantara; RUU tentang Pengelolaan Kekayaan Negara; RUU ASN; RUU Daerah Khusus Jakarta \n',
 'March 2024': 'RUU DKJ; RUU Desa; RUU Masyarakat Adat; RUU PPRT; RUU KIA; RUU EBET; RUU Penyiaran; RUU Perkoperasian; RUU Ketenagakerjaan; RUU Paten; RUU KUHP; RUU MK \n',
 'April 2024': 'RUU Desa; RUU Pengasuhan Anak; RUU Penyiaran; RUU Pernikahan Sesama Jenis; RUU MD3; RUU Daerah Khusus Jakarta; RUU Bahasa Daerah; RUU Kabu

In [ ]:
for month, response in d_responses.items():
    print(month)

    l_ruu = response.strip().replace('\n','').split("; ")
    l_ruu = list(set(l_ruu))
    for ruu in l_ruu:
        print(f"- {ruu}")

January 2024
- RUU BHP
- RUU PHN
- RUU Konservasi
- RUU Desa
- RUU Masyarakat Adat
- RUU KSDAHE
- RUU Lembaga Kepresidenan
- RUU Perlayaran
- RUU DKJ
- RUU Pendanaan Politik
- RUU Penyiaran
- RUU Daerah Khusus Jakarta
- RUU Administrasi Pemerintahan
- RUU Perampasan Aset
- RUU Sisdiknas
- RUU Kesehatan
- RUU Aset Daerah
- RUU PPRT
- RUU tentang Pendanaan Partai Politik
- RUU Udara Bersih
- RUU BUMN
February 2024
- RUU DKJ
- RUU ASN
- RUU Desa
- RUU Pelestarian dan perlindungan Adat Budaya Kerajaan Nusantara
- RUU Penyiaran
- RUU Daerah Khusus Jakarta
- RUU EBET
- RUU Masyarakat Adat
- RUU tentang Pengelolaan Kekayaan Negara
- RUU PPRT
- RUU PHN
- RUU Pilkada
- RUU Perampasan Aset
March 2024
- RUU DKJ
- RUU Desa
- RUU KUHP
- RUU Penyiaran
- RUU EBET
- RUU Masyarakat Adat
- RUU KIA
- RUU PPRT
- RUU MK
- RUU Ketenagakerjaan
- RUU Perkoperasian
- RUU Paten
April 2024
- RUU Pembatasan Rokok
- RUU Masyarakat Hukum Adat
- R
- RUU Pernikahan Sesama Jenis
- RUU Desa
- RUU Provinsi DKJ
- RUU Pel

# spacy

In [ ]:
!pip install spacy_stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 23.9 MB/s eta 0:00:00


In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('id')

INFO:stanza:Downloading default packages for language: id (Indonesian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
import pandas as pd
import numpy as np
import spacy_stanza

# Load the Indonesian Stanza model into SpaCy
nlp = spacy_stanza.load_pipeline("id")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
# Function to extract RUU topic using SpaCy and POS tagging
def extract_ruu_topic(title):
    doc = nlp(title)
    ruu_topic = []
    found_ruu = False

    for token in doc:
        if found_ruu:
            # Stop if we encounter a verb, conjunction, or punctuation
            if token.pos_ in {'NOUN', 'PROPN'}:
                ruu_topic.append(token.text)
            else:
                break
        if token.text == "RUU":
            found_ruu = True
            ruu_topic.append(token.text)

    if len(ruu_topic) > 1:
        return ' '.join(ruu_topic)
    return np.nan

In [ ]:
df_news.loc[df_news['title'].str.contains('RUU Polri')].iloc[2]['title']

'DPR Gelar Paripurna Sahkan 4 Revisi UU Inisiatif, RUU Polri hingga TNI - CNN Indonesia'

In [ ]:
df_news.loc[df_news['title'].str.contains('RUU Perampasan')].iloc[2]['title']

'Ditanya Isu Lobi Ketum Partai soal RUU Perampasan Aset, Ini Jawaban Ganjar - detikNews'

In [ ]:
extract_ruu_topic("Mahfud Sindir RUU Perampasan Aset Mandek di DPR: 6 Bulan Tak Ada Respons - detikNews")

'RUU Perampasan Aset Mandek'

In [ ]:
extract_ruu_topic("Ganjar janji RUU Perampasan Aset jadi prioritas pertama di prolegnas - ANTARA")

'RUU Perampasan Aset'

In [ ]:
extract_ruu_topic("Ditanya Isu Lobi Ketum Partai soal RUU Perampasan Aset, Ini Jawaban Ganjar - detikNews")

'RUU Perampasan Aset'

In [ ]:
extract_ruu_topic("DPR Gelar Paripurna Sahkan 4 Revisi UU Inisiatif, RUU Polri hingga TNI - CNN Indonesia")

'RUU Polri'

In [ ]:
title = 'Ditanya Isu Lobi Ketum Partai soal RUU Perampasan Aset, Ini Jawaban Ganjar - detikNews'
doc = nlp(title)

In [ ]:
for token in doc:
  print(token)
  print(token.pos_)

Ditanya
VERB
Isu
PROPN
Lobi
PROPN
Ketum
PROPN
Partai
PROPN
soal
PROPN
RUU
PROPN
Perampasan
PROPN
Aset
PROPN
,
PUNCT
Ini
DET
Jawaban
NOUN
Ganjar
PROPN
-
PUNCT
detikNews
PROPN


In [ ]:
df_news['ruu_topic'] = df_news['title'].apply(extract_ruu_topic)

## batch

In [ ]:
df_news_sample = df_news.iloc[:1000]

In [ ]:
def extract_ruu_topic(doc):
    ruu_topic = []
    found_ruu = False

    for token in doc:
        if found_ruu:
            # Stop if we encounter a verb, conjunction, or punctuation
            if token.pos_ in {'NOUN', 'PROPN'}:
                ruu_topic.append(token.text)
            else:
                break
        if token.text == "RUU":
            found_ruu = True
            ruu_topic.append(token.text)

    if len(ruu_topic) > 1:
        return ' '.join(ruu_topic)
    return np.nan

# Process titles in batches
batch_size = 100  # Adjust batch size as needed
titles = df_news_sample['title'].tolist()
ruu_topics = []

for i in range(0, len(titles), batch_size):
    batch_titles = titles[i:i+batch_size]
    docs = list(nlp.pipe(batch_titles))
    batch_ruu_topics = [extract_ruu_topic(doc) for doc in docs]
    ruu_topics.extend(batch_ruu_topics)

# Add the results to the dataframe
df_news_sample['ruu_topic'] = ruu_topics

In [ ]:
df_news_sample

,title,description,published date,url,publisher,publisher_link,publisher_title,ruu_topic
0,RUU DKJ Tak Sesuai Harapan Masyarakat Betawi -...,RUU DKJ Tak Sesuai Harapan Masyarakat Betawi ...,2024-01-01 08:00:00,https://news.google.com/rss/articles/CBMilgFBV...,"{'href': 'https://rmol.id', 'title': 'RMOL ID'}",https://rmol.id,RMOL ID,RUU DKJ
1,Menkeu : Pembiayaan Anggaran 2023 Turun Drop -...,Menkeu : Pembiayaan Anggaran 2023 Turun Drop ...,2024-01-02 08:00:00,https://news.google.com/rss/articles/CBMipwFBV...,"{'href': 'https://www.kemenkeu.go.id', 'title'...",https://www.kemenkeu.go.id,Kementerian Keuangan,NaN
2,"Dialog lintas agama, PSI perjuangkan kemudahan...","Dialog lintas agama, PSI perjuangkan kemudahan...",2024-01-02 08:00:00,https://news.google.com/rss/articles/CBMiqAFBV...,"{'href': 'https://www.antaranews.com', 'title'...",https://www.antaranews.com,ANTARA,NaN
3,Eks Hakim MK Arsyad Sanusi Meninggal Dunia - C...,Eks Hakim MK Arsyad Sanusi Meninggal Dunia CN...,2024-01-01 08:00:00,https://news.google.com/rss/articles/CBMiqgFBV...,"{'href': 'https://www.cnnindonesia.com', 'titl...",https://www.cnnindonesia.com,CNN Indonesia,NaN
4,MAKI Heran Harun Masiku Sembunyi Lama Padahal ...,MAKI Heran Harun Masiku Sembunyi Lama Padahal ...,2024-01-02 08:00:00,https://news.google.com/rss/articles/CBMitgFBV...,"{'href': 'https://news.detik.com', 'title': 'd...",https://news.detik.com,detikNews,NaN
...,...,...,...,...,...,...,...,...
995,DPR Optimalkan 45 RUU Prolegnas Jelang Akhir P...,DPR Optimalkan 45 RUU Prolegnas Jelang Akhir P...,2024-05-14 07:00:00,https://news.google.com/rss/articles/CBMilAFBV...,"{'href': 'https://www.rri.co.id', 'title': 'rr...",https://www.rri.co.id,rri.co.id,RUU Prolegnas Jelang Akhir Periode
996,Dewan Pers Tolak Draf RUU Penyiaran karena Din...,Dewan Pers Tolak Draf RUU Penyiaran karena Din...,2024-05-14 07:00:00,https://news.google.com/rss/articles/CBMiyAFBV...,"{'href': 'https://nu.or.id', 'title': 'Islam NU'}",https://nu.or.id,Islam NU,RUU Penyiaran
997,Dewan Pers Tolak Revisi UU Penyiaran karena Me...,Dewan Pers Tolak Revisi UU Penyiaran karena Me...,2024-05-14 07:00:00,https://news.google.com/rss/articles/CBMiwAFBV...,"{'href': 'https://nasional.kompas.com', 'title...",https://nasional.kompas.com,Nasional Kompas.com,NaN
998,Draf RUU Penyiaran: Lembaga Penyiaran Berlangg...,Draf RUU Penyiaran: Lembaga Penyiaran Berlangg...,2024-05-14 07:00:00,https://news.google.com/rss/articles/CBMivgFBV...,"{'href': 'https://nasional.kompas.com', 'title...",https://nasional.kompas.com,Nasional Kompas.com,RUU Penyiaran
